In [1]:
import sys
import copy
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
years = reversed(range(2008,2019))

In [3]:
def formatPlayerName(name):
    try:
        starIndex = name.index("*")
    except ValueError:
        starIndex = sys.maxsize
    try:
        slashIndex = name.index("\\")
    except ValueError:
        slashIndex = sys.maxsize
    return name[:min(starIndex, slashIndex)]
def preprocess(years):
    yearToOriginalData = {}
    finalData = pd.DataFrame()
    for index, year in enumerate(years): 
        data = pd.read_excel(str(year) + "NFLData.xlsx")
        data.columns = data.iloc[0]
        data = data.drop(columns = ["Passing", "Rushing", "Scoring", "Fumbles"])
        data = data.drop([0])
        data.columns = data.iloc[0]
        data = data.drop([1])
        data = data.drop(columns = ["Rk", "GS", "PPR", "DKPt", "VBD", "FDPt", "PosRank", "OvRank", "Tm"])
        data = data[data.FantPos == "WR"]
        data = data[data.G >= 12]
        data = data.dropna()
        data["Player"] = data["Player"].map(formatPlayerName)
        data = data.set_index("Player")
        yearToOriginalData[year] = copy.deepcopy(data)
        if(index != 0):
            for name in data.index: 
                if name in yearToOriginalData[year + 1]["FantPt"]:
                    data["FantPt"][name] = yearToOriginalData[year + 1]["FantPt"][name]
                else:
                    data = data.drop(name)
            data.index = data.index.map(lambda name: name + str(year))
            if(finalData.empty):
                finalData = data
            else:
                finalData = finalData.append(data)
    return finalData

In [4]:
finalData = preprocess(years)
print(finalData)

1                   FantPos Age   G  Tgt  Rec   Yds    Y/R  TD FantPt
Player                                                               
DeAndre Hopkins2017      WR  25  15  174   96  1378  14.35  13    219
Antonio Brown2017        WR  29  14  163  101  1533  15.18   9    220
Keenan Allen2017         WR  25  16  159  102  1393  13.66   6    163
Tyreek Hill2017          WR  23  15  105   75  1183  15.77   7    247
Julio Jones2017          WR  28  16  148   88  1444  16.41   3    213
...                     ...  ..  ..  ...  ...   ...    ...  ..    ...
Brad Smith2008           WR  25  15   20   12    64   5.33   0     34
Sam Aiken2008            WR  28  14   11    8   101  12.63   0     45
Jacoby Jones2008         WR  24  16    5    3    81     27   0     82
Legedu Naanee2008        WR  25  16   11    8    64      8   0     38
Pierre Garcon2008        WR  22  14    4    4    23   5.75   0    102

[773 rows x 9 columns]
